In [2]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional, Callable, Union
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem,
    function_tool,
    FunctionTool,
    handoff,
    ModelSettings
)
load_dotenv()

from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph

import json
import random

In [3]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


In [4]:

class StockData(BaseModel):
    """Model for stock data"""
    price: float = Field(..., description="Current stock price")
    volume: int = Field(..., description="Trading volume")
    pe_ratio: float = Field(..., description="Price-to-earnings ratio")
    market_cap: str = Field(..., description="Market capitalization")

@function_tool
def get_stock_data(symbol: str) -> StockData:
    """Get stock market data for a given symbol"""
    return StockData(**{"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"})

class NewsItem(BaseModel):
    """Model for news items"""
    title: str = Field(..., description="Title of the news article")
    date: str = Field(..., description="Publication date")
    summary: str = Field(..., description="Summary of the article")

@function_tool
def get_news(query: str) -> List[NewsItem]:
    """Get recent news articles about a company"""
    return [
        NewsItem(**
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Q1 Earnings Report",
            "date": "2024-03-21",
            "summary": "Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.",
        }),
        NewsItem(**
        {
            "title": "Tesla Partners with Panasonic",
            "date": "2024-03-22",
            "summary": "Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.",
        }),
        NewsItem(**
        {
            "title": "Tesla Model S Plaid Review",
            "date": "2024-03-23",
            "summary": "The Tesla Model S Plaid receives rave reviews for its performance and technology features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Energy Division Grows",
            "date": "2024-03-24",
            "summary": "Tesla's energy division sees significant growth in solar and battery storage solutions.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Global Expansion Plans",
            "date": "2024-03-25",
            "summary": "Tesla announces plans to expand its manufacturing footprint in Europe and Asia.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Autopilot Update",
            "date": "2024-03-26",
            "summary": "Tesla releases a new update for its Autopilot system, improving safety and navigation features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Stock Performance",
            "date": "2024-03-27",
            "summary": "Tesla's stock shows strong performance in the market, driven by positive investor sentiment.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Sustainability Initiatives",
            "date": "2024-03-28",
            "summary": "Tesla announces new sustainability initiatives aimed at reducing carbon footprint.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Cybertruck Launch Date Announced",
            "date": "2024-03-29",
            "summary": "Tesla announces the official launch date for the highly anticipated Cybertruck.",
        }),
        NewsItem(**
        {
            "title": "Tesla's AI Day 2024 Highlights",
            "date": "2024-03-30",
            "summary": "Tesla's AI Day showcases advancements in artificial intelligence and robotics.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Supercharger Network Expansion",
            "date": "2024-03-31",
            "summary": "Tesla announces plans to expand its Supercharger network across North America.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Model 3 Production Update",
            "date": "2024-03-17",
            "summary": "Tesla increases production capacity for the Model 3 to meet growing demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        }),
        NewsItem(**
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        }),
    ]

class Refund(BaseModel):
    flight_id: str = Field(..., description="Flight ID")
    amount: float = Field(..., description="Refund amount")



In [5]:
stockdata = await get_stock_data.on_invoke_tool(None, json.dumps({"symbol": "TSLA"}))
print(stockdata)
news = await get_news.on_invoke_tool(None, json.dumps({"query": "Tesla"}))
print(news)


price=180.25 volume=1000000 pe_ratio=65.4 market_cap='700B'
[NewsItem(title='Tesla Expands Cybertruck Production', date='2024-03-20', summary='Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.'), NewsItem(title="Tesla's Q1 Earnings Report", date='2024-03-21', summary='Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.'), NewsItem(title='Tesla Partners with Panasonic', date='2024-03-22', summary='Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.'), NewsItem(title='Tesla Model S Plaid Review', date='2024-03-23', summary='The Tesla Model S Plaid receives rave reviews for its performance and technology features.'), NewsItem(title="Tesla's Energy Division Grows", date='2024-03-24', summary="Tesla's energy division sees significant growth in solar and battery storage solutions."), NewsItem(title="Tesla's Global Expansion Plans", date='202

In [6]:
financial_analyst = Agent(
    name="financial_analyst",
    model=model_name,
    tools=[get_stock_data],
    output_type=StockData,
    instructions="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    ### IMPORTANT ###
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a financial analyst.
    You perform analysis of the stock market data.
    you must handoff back to the planner agent."""
)


news_analyst = Agent(
    name="news_analyst",
    model=model_name,
    tools=[get_news],
    output_type=List[NewsItem],
    instructions="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a news analyst.
    You retrieve news articles to analyze.
    you must handoff back to the planner agent.""",

)


writer = Agent(
    name="writer",
    model=model_name,
    tools=[],
    output_type=str,
    instructions="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    The report should be easy to understand for a general audience.
    The report is formatted as a markdown document.
    Always handoff back to planner when writing is complete.""",
    handoff_description="""You are a writer.
    You summary a summary of the analysis results.
    you must handoff back to the planner agent."""
)



try:
    draw_graph(financial_analyst, filename="viz/11_planner_financial_analyst.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [7]:
msg = "what is the TSLA stock, write a report"
input_items : list[TResponseInputItem] = [{"content" : msg, "role": "user"}]

In [8]:
# with trace(workflow_name="financial_analyst", group_id="mdsi") as tr:
#     print(f"Tracing url: {get_trace_url(tr)}")

#     result = await Runner.run(
#         financial_analyst,
#         input=input_items,
#         run_config=RunConfig(
#             model=model_name,
#             model_provider=GITHUB_MODEL_PROVIDER
#         )

#     )

#     print(result)

In [9]:
planner_assistant = Agent(
    name="planner",
    model=model_name,
    handoffs=[financial_analyst,news_analyst, writer],
    instructions="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
    handoff_description="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents
    you are responsible for generating the final output.,
    """,

)

planner_assistant.handoffs = [writer]
planner_assistant.tools = [
    financial_analyst.as_tool(
        tool_name="financial_analyst",
        tool_description="Analyze stock market data using the get_stock_data tool. Provide insights on financial metrics."
    ),
    news_analyst.as_tool(
        tool_name="news_analyst",
        tool_description="Gather and analyze relevant news using the get_news tool. Summarize key market insights from news."
    ),

]
# financial_analyst.handoffs = [handoff(planner_assistant)]
# news_analyst.handoffs = [handoff(planner_assistant)]
# writer.handoffs = [handoff(planner_assistant)]

try:
    draw_graph(planner_assistant, filename="viz/11_planner_financial_planner_as_tools.gv")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    draw_graph(writer, filename="viz/11_planner_writer.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [10]:
with trace(workflow_name="financial_planner", group_id="financial_planner") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    result = await Runner.run(
        planner_assistant,
        input=input_items,
        run_config=RunConfig(
            model=model_name,
            model_provider=GITHUB_MODEL_PROVIDER,
            model_settings=ModelSettings(**{
                # "temperature": 0.5,
                "max_tokens": 5000,
                # "top_p": 1.0,
                # "frequency_penalty": 0.0,
                # "presence_penalty": 0.0,
                # "stop": ["\n\n"]
                }, # Adjust the model settings as needed
        )
        )


    )

    print(result)

Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_403faa73e3a14b80893d64b8b0db5547


RunResult:
- Last agent: Agent(name="writer", ...)
- Final output (str):
    # Tesla, Inc. (TSLA) Stock Report
    
    ## Overview
    Tesla, Inc. (NASDAQ: TSLA) is a leading electric vehicle and clean energy company known for its innovative technology and expanding product line. As of the latest available data, the stock price stands at **$180.25**, with a market capitalization of **$700 billion**. The company's trading volume has reached approximately **1,000,000 shares**, and it has a Price-to-Earnings (P/E) ratio of **65.4**, indicating a high valuation compared to earnings.
    
    ## Recent News Highlights
    Tesla has been in the news recently for several significant developments that may impact its stock performance and investor sentiment:
    
    1. **Cybertruck Production Expansion**: Tesla is increasing production capacity for its Cybertruck at Gigafactory Texas in response to expected strong demand.
    
    2. **Positive Earnings Forecast**: Analysts anticipate strong 

In [11]:
print(result.last_agent.model_settings.max_tokens)

None


In [12]:
planner_assistant.model_settings

ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None)

In [13]:
result.raw_responses[-1].usage

Usage(requests=1, input_tokens=1447, output_tokens=707, total_tokens=2154)

In [14]:
display(Markdown(result.final_output))

# Tesla, Inc. (TSLA) Stock Report

## Overview
Tesla, Inc. (NASDAQ: TSLA) is a leading electric vehicle and clean energy company known for its innovative technology and expanding product line. As of the latest available data, the stock price stands at **$180.25**, with a market capitalization of **$700 billion**. The company's trading volume has reached approximately **1,000,000 shares**, and it has a Price-to-Earnings (P/E) ratio of **65.4**, indicating a high valuation compared to earnings.

## Recent News Highlights
Tesla has been in the news recently for several significant developments that may impact its stock performance and investor sentiment:

1. **Cybertruck Production Expansion**: Tesla is increasing production capacity for its Cybertruck at Gigafactory Texas in response to expected strong demand.

2. **Positive Earnings Forecast**: Analysts anticipate strong earnings in Tesla's upcoming quarterly report, driven by record vehicle deliveries.

3. **Partnership with Panasonic**: The company is enhancing its battery technology through a new partnership with Panasonic, which could further improve the performance of its electric vehicles.

4. **Model S Plaid Reviews**: The Tesla Model S Plaid has received excellent reviews for its performance and advanced features, enhancing the brand's reputation.

5. **Growth in Energy Sector**: Tesla's energy division is experiencing significant growth, particularly in solar energy and battery storage solutions, which are important for the company's diversification.

6. **Global Expansion**: Tesla is planning to expand its manufacturing presence in Europe and Asia, likely aiming to capture more of the global EV market.

7. **Autopilot Features Update**: The recent update to Tesla's Autopilot system has improved safety and navigation features, contributing to its competitive edge in autonomous driving technology.

8. **Stock Performance Trends**: The stock has shown a positive performance recently, buoyed by optimistic investor sentiment following these developments.

9. **Sustainability Initiatives**: Tesla has announced new sustainability initiatives aimed at further reducing its carbon footprint, aligning with its mission to promote clean energy.

10. **Upcoming Cybertruck Launch**: The highly anticipated launch date for the Cybertruck has been officially announced, which is expected to generate significant consumer interest.

11. **AI Day 2024**: Tesla's AI Day showcased new advancements in artificial intelligence and robotics, emphasizing the company's focus on integrating cutting-edge technology into its products.

12. **Supercharger Network Expansion**: Plans for expanding Tesla's Supercharger network across North America have been revealed, enhancing the convenience of owning a Tesla vehicle.

13. **Model 3 Production Increase**: Production capacity for the Model 3 has been increased to meet growing consumer demand.

14. **FSD Beta Improvements**: The latest update to the Full Self-Driving (FSD) beta program has demonstrated substantial improvements, particularly in urban navigation.

15. **Model Y Sales Success**: Tesla's Model Y has become the best-selling electric vehicle globally, indicating strong market penetration.

## Conclusion
Tesla, Inc. continues to show promise with robust growth strategies, innovative technology, and a solid market presence. Investors should closely monitor upcoming earnings reports and major product launches, as these factors will likely influence the stock's performance in the near future.

---

This report on Tesla, Inc. (TSLA) has been compiled for your understanding. Please make sure to review the contents thoroughly, and let me know if you require any additional information. 

In [15]:
tr.finish()

In [16]:
mm = GITHUB_MODEL_PROVIDER.get_model(model_name)


In [17]:
mm.

SyntaxError: invalid syntax (1051154210.py, line 1)